<a href="https://colab.research.google.com/github/HongJaeKwon/kimhae_fire_detection/blob/master/hjk_fire_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
# /gdrive/My Drive/ (폴더명)

Mounted at /gdrive


In [0]:
import numpy as np
import pandas as pd
import warnings
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from  sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import losses
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, Activation, Flatten, Dense, UpSampling2D, Reshape, Lambda, Input,BatchNormalization, Dropout, LeakyReLU
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, Iterator
from tensorflow.python.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.python.keras.losses import mean_absolute_error
warnings.filterwarnings(action='ignore') 

In [0]:
all_DF= pd.read_csv('/gdrive/My Drive/김해화재/all_DF.csv',index_col=0)
# train :59199
# val : 6898
# test : 2957

In [30]:
all_DF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,LABEL
0,0.200443,0.000121,0.043593,0.002841,0.000000,0.009122,0.039669,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.059644,0.000081,0.006703,0.002841,0.000000,0.066623,0.040868,0.000099,0.000101,0.000105,0.000094,0.000098,0.000099,0.000107,0.000106,0.000129,0.000175,0.000196,0.000278,0.000265,0.000318,0.000224,0.00023,0.000228,0.000205,0.000201,0.000213,0.000228,0.000312,0.000306,0.000303,0.000357,0.000415,0.000344,0.000368,0.000373,0.000273,0.001115,0.001119,0.001137,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.220430,0.001019,0.038322,0.000000,0.166667,0.013439,0.042426,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,0.007205,0.000085,0.044794,0.001420,0.130952,0.010669,0.034756,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.221068,0.000000,0.043593,0.001420,0.000000,0.055872,0.040868,0.000099,0.000101,0.000105,0.000094,0.000098,0.000099,0.000107,0.000106,0.000129,0.000175,0.000196,0.000278,0.000265,0.000318,0.000224,0.00023,0.000228,0.000205,0.000201,0.000213,0.000228,0.000312,0.000306,0.000303,0.000357,0.000415,0.000344,0.000368,0.000373,0.000273,0.001115,0.001119,0.001137,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [0]:
train_DF=all_DF[:59199]
val_DF=all_DF[59199:59199+6898]
test_DF=all_DF[all_DF['LABEL']==-1]

train_DF_Y=train_DF['LABEL']
val_DF_Y=val_DF['LABEL']
train_DF_normal=train_DF[train_DF['LABEL']==0]
train_DF_fire=train_DF[train_DF['LABEL']==1]

train_DF=train_DF.drop('LABEL', axis = 1)
val_DF=val_DF.drop('LABEL', axis = 1)
test_DF=test_DF.drop('LABEL', axis = 1)

train_DF_normal=train_DF_normal.drop('LABEL', axis = 1)
train_DF_fire=train_DF_fire.drop('LABEL', axis = 1)

train_DF_1=train_DF[:30000]
train_DF_Y_1=train_DF_Y[:30000]
train_DF_2=train_DF[30000:]
train_DF_Y_2=train_DF_Y[30000:]

In [0]:
col_list=train_DF.columns.to_list()


In [0]:
stack_DF=pd.DataFrame()
stack_DF_val=pd.DataFrame()
stack_DF_test=pd.DataFrame()
lda=LinearDiscriminantAnalysis()
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=7)
rf_clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=0)
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=0)
lr = LogisticRegression(random_state=0, solver='lbfgs')
gnb = GaussianNB()
svc = LinearSVC(random_state=0, tol=1e-5)
qda = QuadraticDiscriminantAnalysis()

In [15]:
# lda.fit(train_DF,train_DF_Y)
# y_pred=lda.predict(val_DF)
# print ("accuracy_score : " , accuracy_score(val_DF_Y, y_pred))
# print ("f1_score : " , f1_score(val_DF_Y, y_pred))
# print ("precision_score : " , precision_score(val_DF_Y, y_pred))
# print ("recall_score : " , recall_score(val_DF_Y, y_pred))

accuracy_score :  0.7368802551464193
f1_score :  0.41205053449951407
precision_score :  0.3504132231404959
recall_score :  0.5


In [0]:
np.random.seed(0)
for i in range(20):
    print(i)
    rnd_list=np.random.choice(col_list, 200, replace=False)
    
    lda=lda.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['lda'+str(i)]=np.reshape(lda.predict_proba(train_DF_2[rnd_list])[:,1],(-1,))
    stack_DF_val['lda'+str(i)]=np.reshape(lda.predict_proba(val_DF[rnd_list])[:,1],(-1,))
    stack_DF_test['lda'+str(i)]=np.reshape(lda.predict_proba(test_DF[rnd_list])[:,1],(-1,))
    print("lda OK")

    decision_tree = decision_tree.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['dt'+str(i)]=np.reshape(decision_tree.predict_proba(train_DF_2[rnd_list])[:,1],(-1,))
    stack_DF_val['dt'+str(i)]=np.reshape(decision_tree.predict_proba(val_DF[rnd_list])[:,1],(-1,))
    stack_DF_test['dt'+str(i)]=np.reshape(decision_tree.predict_proba(test_DF[rnd_list])[:,1],(-1,))
    print("decision_tree OK")

    rf_clf = rf_clf.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['rf'+str(i)]=np.reshape(rf_clf.predict_proba(train_DF_2[rnd_list])[:,1],(-1,))
    stack_DF_val['rf'+str(i)]=np.reshape(rf_clf.predict_proba(val_DF[rnd_list])[:,1],(-1,))
    stack_DF_test['rf'+str(i)]=np.reshape(rf_clf.predict_proba(test_DF[rnd_list])[:,1],(-1,))
    print("rf_clf OK")

    xgb_model = xgb_model.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['xgb'+str(i)]=np.reshape(xgb_model.predict_proba(train_DF_2[rnd_list])[:,1],(-1,))
    stack_DF_val['xgb'+str(i)]=np.reshape(xgb_model.predict_proba(val_DF[rnd_list])[:,1],(-1,))
    stack_DF_test['xgb'+str(i)]=np.reshape(xgb_model.predict_proba(test_DF[rnd_list])[:,1],(-1,))
    print("xgb_model OK")

    # cnn_model.fit(train_DF_1[rnd_list].as_matrix(),train_DF_Y_1.as_matrix(),epochs=50,verbose=0)
    # stack_DF['cnn'+str(i)]=np.reshape(cnn_model.predict(train_DF_2[rnd_list].as_matrix()),(-1,))
    # stack_DF_val['cnn'+str(i)]=np.reshape(cnn_model.predict(val_DF[rnd_list].as_matrix()),(-1,))
    # stack_DF_test['cnn'+str(i)]=np.reshape(cnn_model.predict(test_DF[rnd_list].as_matrix()),(-1,))
    # print("cnn_model OK")

    lr = lr.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['lr'+str(i)]=np.reshape(lr.predict_proba(train_DF_2[rnd_list])[:,1],(-1,))
    stack_DF_val['lr'+str(i)]=np.reshape(lr.predict_proba(val_DF[rnd_list])[:,1],(-1,))
    stack_DF_test['lr'+str(i)]=np.reshape(lr.predict_proba(test_DF[rnd_list])[:,1],(-1,))
    print("lr OK")

    gnb = gnb.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['gnb'+str(i)]=np.reshape(gnb.predict_proba(train_DF_2[rnd_list])[:,1],(-1,))
    stack_DF_val['gnb'+str(i)]=np.reshape(gnb.predict_proba(val_DF[rnd_list])[:,1],(-1,))
    stack_DF_test['gnb'+str(i)]=np.reshape(gnb.predict_proba(test_DF[rnd_list])[:,1],(-1,))
    print("gnb OK")

    svc = svc.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['svc'+str(i)]=np.reshape(svc.decision_function(train_DF_2[rnd_list]),(-1,))
    stack_DF_val['svc'+str(i)]=np.reshape(svc.decision_function(val_DF[rnd_list]),(-1,))
    stack_DF_test['svc'+str(i)]=np.reshape(svc.decision_function(test_DF[rnd_list]),(-1,))
    print("svc OK")

    qda = qda.fit(train_DF_1[rnd_list], train_DF_Y_1)
    stack_DF['qda'+str(i)]=np.reshape(qda.predict_proba(train_DF_2[rnd_list])[:,1],(-1,))
    stack_DF_val['qda'+str(i)]=np.reshape(qda.predict_proba(val_DF[rnd_list])[:,1],(-1,))
    stack_DF_test['qda'+str(i)]=np.reshape(qda.predict_proba(test_DF[rnd_list])[:,1],(-1,))
    print("qda OK")


In [71]:
stack_DF

,lda0,dt0,rf0,xgb0,lr0,gnb0,svc0,qda0,lda1,dt1,rf1,xgb1,lr1,gnb1,svc1,qda1,lda2,dt2,rf2,xgb2,lr2,gnb2,svc2,qda2,lda3,dt3,rf3,xgb3,lr3,gnb3,svc3,qda3,lda4,dt4,rf4,xgb4,lr4,gnb4,svc4,qda4,...,lda15,dt15,rf15,xgb15,lr15,gnb15,svc15,qda15,lda16,dt16,rf16,xgb16,lr16,gnb16,svc16,qda16,lda17,dt17,rf17,xgb17,lr17,gnb17,svc17,qda17,lda18,dt18,rf18,xgb18,lr18,gnb18,svc18,qda18,lda19,dt19,rf19,xgb19,lr19,gnb19,svc19,qda19
0,0.025579,0.000000,0.000000,0.000369,0.155902,7.164993e-35,-0.660146,1.0,0.032354,0.000000,0.0000,0.000417,0.082569,8.759612e-44,-0.748852,1.000000e+00,0.023645,0.000000,0.00,0.000479,0.045568,6.446619e-46,-0.870058,1.0,0.012703,0.000000,0.00,0.000868,0.050731,1.469251e-46,-0.888785,1.000000e+00,0.009556,0.000000,0.00,0.000334,0.025394,1.782251e-53,-1.011402,1.000000,...,0.028060,0.000000,0.00,0.000509,0.052219,5.590531e-40,-0.813885,1.0,0.017986,0.000000,0.00,0.000449,0.030859,4.317541e-68,-0.940212,1.000000e+00,0.018442,0.000000,0.00,0.000475,0.046954,1.120105e-41,-0.873030,1.0,0.017969,0.000000,0.0000,0.000512,0.046952,4.883185e-24,-0.894689,1.0,0.036415,0.000000,0.00,0.000470,0.201324,7.655447e-41,-0.582630,1.0
1,0.012422,0.000000,0.000000,0.000453,0.041861,1.939163e-37,-0.958554,1.0,0.008824,0.000000,0.0000,0.000421,0.010462,3.249030e-54,-1.224651,1.000000e+00,0.010393,0.000000,0.00,0.000451,0.009599,3.666272e-56,-1.221856,1.0,0.014898,0.000000,0.00,0.000831,0.032047,2.400643e-54,-0.938898,1.000000e+00,0.011564,0.000000,0.00,0.000746,0.024448,3.113082e-54,-0.984775,1.000000,...,0.007664,0.000000,0.00,0.000586,0.006061,9.671554e-51,-1.302707,1.0,0.008769,0.000000,0.00,0.000561,0.016520,6.831459e-69,-1.084714,1.000000e+00,0.032658,0.000000,0.00,0.000535,0.053991,2.105614e-49,-0.808468,1.0,0.016806,0.000000,0.0000,0.000522,0.038357,6.097503e-25,-0.925342,1.0,0.014163,0.000000,0.00,0.000452,0.033807,2.026934e-42,-0.974694,1.0
2,0.006281,0.000000,0.000000,0.000387,0.036937,3.929273e-38,-0.981228,1.0,0.003423,0.000000,0.0000,0.000358,0.006976,1.749374e-53,-1.350849,2.806101e-20,0.005332,0.000000,0.00,0.000404,0.012241,2.868574e-52,-1.224442,1.0,0.003980,0.000000,0.00,0.000464,0.014783,5.189812e-55,-1.158613,1.000000e+00,0.008476,0.000000,0.00,0.000631,0.024107,1.373083e-53,-1.026798,1.000000,...,0.003507,0.000000,0.00,0.000299,0.007479,2.175963e-45,-1.344202,1.0,0.003417,0.000000,0.00,0.000558,0.008743,1.024878e-72,-1.266840,1.000000e+00,0.006861,0.000000,0.00,0.000388,0.026182,1.201739e-46,-1.029640,1.0,0.014195,0.000000,0.0000,0.000552,0.087536,6.312186e-22,-0.817599,1.0,0.004333,0.000000,0.00,0.000418,0.025622,8.852382e-44,-1.071287,1.0
3,0.007348,0.000000,0.000000,0.000353,0.056728,3.191137e-39,-0.891001,1.0,0.016455,0.000000,0.0000,0.000446,0.047919,6.340791e-47,-0.896257,1.000000e+00,0.014382,0.000000,0.00,0.000468,0.046081,5.355685e-47,-0.904110,1.0,0.005215,0.000000,0.00,0.000520,0.025087,1.066780e-50,-1.060831,1.000000e+00,0.006372,0.000000,0.00,0.000478,0.019275,3.998644e-54,-1.076066,1.000000,...,0.015534,0.000000,0.00,0.000408,0.053916,4.438314e-38,-0.875036,1.0,0.007331,0.000000,0.00,0.000463,0.023677,3.419409e-66,-1.048423,1.000000e+00,0.009574,0.000000,0.00,0.000525,0.046745,8.330233e-41,-0.927649,1.0,0.013225,0.000000,0.0000,0.000441,0.091529,9.891037e-22,-0.810075,1.0,0.012222,0.000000,0.00,0.000385,0.093663,2.270334e-43,-0.791461,1.0
4,0.013422,0.000000,0.000000,0.000416,0.090125,3.495921e-37,-0.784273,1.0,0.008115,0.000000,0.0000,0.000494,0.023279,6.386243e-46,-1.074764,1.000000e+00,0.004647,0.000000,0.00,0.000322,0.007180,8.526065e-56,-1.305265,1.0,0.004777,0.000000,0.00,0.000471,0.013912,7.625941e-56,-1.179298,1.000000e+00,0.012732,0.000000,0.00,0.000565,0.034582,6.873738e-53,-0.943795,0.963754,...,0.005710,0.000000,0.00,0.000310,0.007482,2.603641e-47,-1.289573,1.0,0.007152,0.000000,0.00,0.000459,0.012109,4.158766e-68,-1.152512,1.000000e+00,0.011761,0.000000,0.00,0.000442,0.028032,5.524031e-42,-0.976811,1.0,0.007088,0.000000,0.0000,0.000324,0.013738,1.588935e-32,-1.123417,1.0,0.009152,0.000000,0.00,

In [72]:
stack_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
stack_model.fit(stack_DF, train_DF_Y_2)
y_pred=stack_model.predict(stack_DF_val)
print ("accuracy_score : " , accuracy_score(val_DF_Y, y_pred))
print ("f1_score : " , f1_score(val_DF_Y, y_pred))
print ("precision_score : " , precision_score(val_DF_Y, y_pred))
print ("recall_score : " , recall_score(val_DF_Y, y_pred))

accuracy_score :  0.7957378950420412
f1_score :  0.5126253891387064
precision_score :  0.4576899320568252
recall_score :  0.5825471698113207


In [0]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(ratio='auto', kind='regular')
# X_resampled, y_resampled = sm.fit_sample(stack_DF,list(train_DF_Y_2))

In [67]:
# stack_model.fit(X_resampled, y_resampled)
# y_pred=stack_model.predict(stack_DF_val.as_matrix())
# print ("accuracy_score : " , accuracy_score(val_DF_Y, y_pred))
# print ("f1_score : " , f1_score(val_DF_Y, y_pred))
# print ("precision_score : " , precision_score(val_DF_Y, y_pred))
# print ("recall_score : " , recall_score(val_DF_Y, y_pred))

accuracy_score :  0.5278341548274862
f1_score :  0.4178731009830205
precision_score :  0.27041406430719406
recall_score :  0.9190251572327044


In [0]:
y_pred=stack_model.predict(stack_DF_test)
sub = pd.read_csv('/gdrive/My Drive/김해화재/PJT002_submission.csv')
sub['fr_yn'] = y_pred
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})
sub.to_csv('/gdrive/My Drive/김해화재/PJT002_submission_hjk_stack5.csv', index=False)